Attempt to load an already existing Pinecone Index and create a pipeline on it

In [1]:
import os
import pickle
import logging
import time
from haystack.document_stores import PineconeDocumentStore
from haystack import Document
from haystack.nodes import PreProcessor, EmbeddingRetriever, Seq2SeqGenerator, TransformersSummarizer, FARMReader
from haystack.pipelines import GenerativeQAPipeline, ExtractiveQAPipeline, SearchSummarizationPipeline

In [2]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.WARNING)


In [3]:
MODEL_LOCATION = "../twig_otherverse_parahumans_adapted/"
PALE_CHAPTERS_FNAME = "../data/chapter_fmt_list.pkl"

In [4]:
with open(PALE_CHAPTERS_FNAME,'rb') as f:
    all_chapters = pickle.load(f)
chapters = [i for i in all_chapters if int(i['meta']['arc_number']) < 3]
print(f"Testing with {len(chapters)} chapters")

Testing with 30 chapters


In [5]:
document_store = PineconeDocumentStore(
    index = 'all-pale-docs',
    api_key='6dcb03dd-509b-4ad3-8342-b705027cb07c',
    similarity='cosine',
    duplicate_documents='skip',
    embedding_field="emb",
    embedding_dim=768,
    metadata_config = {'indexed':['series_chapter_number','chapters','title']}
)

In [6]:
retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model=MODEL_LOCATION,
    model_format="sentence_transformers",
    max_seq_len=500,
    progress_bar=True,
)

In [7]:
generator = Seq2SeqGenerator(model_name_or_path="vblagoje/bart_lfqa", num_beams=8, max_length=500)
# Eventually with want to tweak the above
pipe = GenerativeQAPipeline(generator, retriever) # We specify the params later

In [8]:
TEST_QUESTION = "Who is Avery Kelly?"

TEST_OUTPUT = pipe.run(
                query = TEST_QUESTION,
                params = {
                    "Retriever": {"top_k":10},
                    # "Generator": {"max_length": maxlength, "num_beams": nbeam}
                }
            )

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
TEST_OUTPUT.keys()